\---

# CSCI 3202, Fall 2024
# FINAL PROJECT

<br> 

### Your name: Gavin Hanville

<br> 

# Mancala Game Implementation

In this assignment, you are tasked with implementing various functions for a Mancala game. The game is played on a board with specific rules, and you will need to implement the core game logic by completing the `play`, `valid_move`, and `winning_eval` functions. You are provided with the `init` and `display_board` functions. The assignment is divided into two parts:

## Mancala rules to be followed 
**(there are few modifications from the original game, please read this before writing the code)**

- On every turn, select a pit and distribute its stones in a counter-clockwise direction.
    - If the last stone lands in the player's mancala, in an opponent's pit, or in one of the player's non-empty pits, no further action is taken, and the current player's turn ends.
    - If the last stone lands in the current player's empty pit and the opposite pit on the opponent's side has some stones, collect all those stones, including the one that just landed, and place them into the current player's mancala.

- If either player's pits are entirely empty, the game concludes. The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.

## Part 1: Small Board (3 Pits of 2 Stones each) (60 points)

For the first part of the assignment, students will work on a small Mancala board. The board consists of 3 pits, each initially containing 2 stones. The students need to implement the following:

1. **play**: Implement the `play` function to allow players to take turns and make moves. The function should correctly distribute stones according to the specified game rules. The game should also switch between players after each play. **(20 points)**

2. **valid_move**: Implement the `valid_move` function to ensure that a player's chosen move is valid. It should check if the selected pit is not empty and falls within the allowed pit range. **(20 points)**

3. **winning_eval**: Implement the `winning_eval` function to determine when the game is over and which player wins. The game ends when any player's pits are all empty. The winner is the player with the most stones in their mancala. If both mancalas have the same number of stones, it's a tie. **(20 points)**

Students should test their code by playing a sequence of moves starting with Player 1: 1, 2, 3, 2, 1. \
So, it would be P1 picks pit 1, P2 picks pit 2, P1 picks pit 3...and so on.
The pits are 1-indexed when displaying and picking to make a move.

Pick the pit irrespective of its validity, and print invalid move message if chosen pit is empty or invalid.

The output generated by this experiment must match the expected output given below.

## Part 2: Play Against a Random Player (6 Pits of 4 Stones each) (40 points)

In the second part of the assignment, students will extend their implementation to a larger board. The board consists of 6 pits with 4 stones in each pit. In addition to the `play`, `valid_move`, and `winning_eval` functions, students need to create a random move generator for a random player. This random player selects a random valid pit with stones to make a move. The following steps are involved in creating the random move generator:

1. **Random Move Generator**: Define the `random_move_generator` that selects a random pit from the available non-empty pits for the random player. The random player should choose a move based on these criteria. \
Set the 'seed' value to ensure that the generated values remain consistent and reproducible when grading.

You may refer to these links: [How to generate random integers in Python](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/#:~:text=Random%20integer%20values%20can%20be,for%20the%20generated%20integer%20values.), [How to use seed in Python random](https://www.w3schools.com/python/ref_random_seed.asp)


The objective is to play up to **10** moves in total (5 moves by student, 5 moves by random player), allowing the students to verify whether their code correctly implements the Mancala game logic. **(20 points for correct implementation of Random Move Generator)**

The output submitted should reflect the state of the board and the moves played. **(10 points for playing game, 10 points for printing out results)**

**Please make sure to call the `display_board` function after each move for both the parts and run all the cells before submitting**

In [1]:
import os
os.chdir('/home/jovyan/CSCI3202/Final Project/aima-python')
print(os.getcwd())  # Verify the change

from probability4e import *
from games4e import *
from utils import print_table
import random as random
random.seed(109)

/home/jovyan/CSCI3202/Final Project/aima-python


In [79]:
#Logger Var
DEBUG = False
if DEBUG:
    print(f"DEBUG: Message")


#Step 2: Improve Readability
    
# Board class: Manages the game board's state and operations (e.g., distributing stones, checking empty pits).
class Board:
    def __init__(self, pits_per_player, stones_per_pit):
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        self._initialize_board()
   
    def _initialize_board(self):
        self.board = [self.stones_per_pit] * ((self.pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
    
    def pit_to_board_index(self, pit_number, current_player):
        """
        Helper function to simplify relationship between pit and board index
        """
        if current_player == 1:
            return pit_number - 1 # P1's pits start at index 0
        else:
            index = self.p2_pits_index[0] + (pit_number - 1)
            return index # P2's pits in reverse
        
    def distribute_stones(self, start_index, stones, skip_index, current_player):
        #Counter-Clockwise Stone Distribution
        board_index = self.pit_to_board_index(start_index, current_player)
        self.board[board_index] = 0 # Empty selected pit at beginning
        index = board_index
        #print(f"DEBUG: Current index: {index}, Stones remaining: {stones}") # TEST
        if not current_player: stones -=1 # Dirty fix to extra stones problem
        while stones > 0:
            #TO-DO: 
            #If the last stone lands in the current player's empty pit and the opposite pit on the opponent's side has some stones, 
            #collect all those stones, including the one that just landed, and place them into the current player's mancala.
            
            index = (index + 1) % len(self.board)
            # Skip Mancala pits
            if (current_player and index == self.p2_mancala_index) or (not current_player and index == self.p1_mancala_index):
                #print(f"DEBUG: Skipping Mancala at index {index}")
                continue
            self.board[index] += 1
            stones -= 1
        
        #print(f"DEBUG: Final index after distribution: {index}")
        return index #verify final index
        
        
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))

    
    def valid_move(self, pit, current_player):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        board_index = self.pit_to_board_index(pit, current_player)
        if board_index < 0 or board_index >= len(self.board):
            print(f"Invalid move (Pit:{pit})")
            return False
        if self.board[board_index] == 0:
            print(f"Invalid move (Pit:{pit})")
            return False
        return True
        
        pass
    
    
# Game class: Handles player interactions, turn management, and win/loss evaluation.    
class Game:
    def __init__(self, pits_per_player=6, stones_per_pit = 5, mode = 0):
        self.board = Board(pits_per_player, stones_per_pit)
        self.current_player = 1
        self.player_mode = mode
        self.moves = []
        return
    
    def get_player_input(self):
        """
        Helper function for user input
        """
        player = "1" if self.current_player == 1 else "2"
        while True:
            try:
                pit = int(input(f"Player {player}, choose your pit (1-{self.board.pits_per_player}): "))
                if 1 <= pit <= self.board.pits_per_player:
                    if self.board.valid_move(pit, self.current_player):
                        return pit
                    else:
                        print(f"({pit}) is empty or invalid. Please try a different pit number...")
                else:
                    print(f"({pit}) is an Invalid Pit number...Please choose a number within range (1-{self.board.pits_per_player})")
            except ValueError:
                print("Invalid input. Please enter an integer!")
                
    def get_cpu_input(self):
        """
        Generate Valid CPU move
        """
        turn = '1' if self.current_player == 1 else '2'
        
        pit = self.random_move_generator()
        if pit is not None:
            return pit
        else: print(f"Player {turn} has no valid moves. Skipping turn.")
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        turn = '1' if self.current_player == 1 else '2'
        
        # Create a list of valid pits
        valid_pits = [pit for pit in range(1, self.board.pits_per_player + 1) if self.board.valid_move(pit, self.current_player)]

        # Sanity print
        print(f"Valid pits for Player {turn}: {valid_pits}")

        if not valid_pits:
            print(f"No valid moves for Player {turn}")
            return None  # Indicate that no move can be made

        # Select a pit at random
        random_pit = random.choice(valid_pits)
        board_index = self.board.pit_to_board_index(random_pit, self.current_player)
        stones = self.board.board[board_index]

        return random_pit        
    
    def play(self):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        #(PVP)
        if self.player_mode == 0:
            while not self.winning_eval():
                self.board.display_board()
                self.print_current_player()
                pit = self.get_player_input()
                self.play_turn(pit)
                
        
        #(PvCPU)
        if self.player_mode == 1:
            while not self.winning_eval():
                self.board.display_board()
                self.print_current_player
                if self.current_player:
                    self.print_current_player()
                    pit = self.get_player_input()
                    self.play_turn(pit)
                else:
                    self.print_current_player()
                    pit = self.get_cpu_input()
                    self.play_turn(pit)
                    
        #(CPUvCPU)
        if self.player_mode == 2:
            #TESTING MOVE COUNT CAP: 10
            while not self.winning_eval():
            #x = 0
            #while x < 10:
                self.board.display_board()
                print(self.board.board)
                self.print_current_player()
                pit = self.get_cpu_input()
                self.play_turn(pit)
                #x += 1
        
        
    def play_turn(self, pit):
        turn = '1' if self.current_player == 1 else '2'
        stones = self.board.board[self.board.pit_to_board_index(pit, self.current_player)]
        print(f"Player: {turn} chose Pit: ({pit}) with Initial stones: {stones}") # Test!
        # Check if selected move is valid
        if not self.board.valid_move(pit, self.current_player):
            return #Invalid move, take no action

        # Distribute stones and catch final index for special rules
        final_index = self.board.distribute_stones(pit, self.board.board[pit], self.board.p1_mancala_index, self.current_player)
        self.apply_special_rules(final_index+1)

        # Update moves list with valid moves
        self.moves.append((turn, pit))

        #Switch Turns
        self.switch_player()
        
    def print_current_player(self):
        if self.current_player == 1: print("Current Player: Player 1")
        else: print("Current Player: Player 2")
        
    def switch_player(self):
        self.current_player = not self.current_player
   
    def apply_special_rules(self, last_index):
        """
        Apply Special CSCI3202 Ruleset after a move...
        If the last stone lands in the current player's empty pit,
        collect all stones if they exist in the opponent's pit--opposite the empty pit,
        including the one that just landed in that empty pit
        """
        # Initialize captured_stones
        captured_stones = 0

        # Initialize current player's range of pits
        if self.current_player == 1:
            own_pits = range(self.board.p1_pits_index[0], self.board.p1_pits_index[1] + 1)
            opponent_pits = range(self.board.p2_pits_index[0], self.board.p2_pits_index[1] + 1)
            mancala_index = self.board.p1_mancala_index
        else:
            own_pits = range(self.board.p2_pits_index[0], self.board.p2_pits_index[1] + 1)
            opponent_pits = range(self.board.p1_pits_index[0], self.board.p1_pits_index[1] + 1)
            mancala_index = self.board.p2_mancala_index

        # Check if the last stone landed in current player's pits
        if last_index in own_pits:
            # Check if the pit now has exactly one stone (it was empty before)
            if self.board.board[last_index] == 1:
                print(f"last_index")
                # Calculate the opposite pit index using pit_to_board_index
                pit_number = last_index + 1  # Convert index back to pit number (1-based)
                if self.current_player == 1:
                    opposite_pit_number = self.board.pits_per_player - pit_number + 1
                    opposite_index = self.board.pit_to_board_index(opposite_pit_number, 0)  # Opponent is Player 2 (0)
                else:
                    opposite_pit_number = self.board.pits_per_player - pit_number + 1
                    opposite_index = self.board.pit_to_board_index(opposite_pit_number, 1)  # Opponent is Player 1 (1)

                # Check if the opposite pit has stones
                if self.board.board[opposite_index] > 0:
                    # Capture stones
                    captured_stones = self.board.board[opposite_index] + self.board.board[last_index]
                    self.board.board[opposite_index] = 0
                    self.board.board[last_index] = 0

                    # Place the captured stones into the current player's Mancala
                    self.board.board[mancala_index] += captured_stones

                    print(f"Player {1 if self.current_player == 1 else 2} captures {captured_stones} stones!")

            
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        # write your code here       
        p1_empty = all(stone == 0 for stone in self.board.board[self.board.p1_pits_index[0]:self.board.p1_pits_index[1]+1])
        p2_empty = all(stone == 0 for stone in self.board.board[self.board.p2_pits_index[0]:self.board.p2_pits_index[1]+1])

        if p1_empty or p2_empty:
            #Collect remaining stones into respective Mancalas
            self.board.board[self.board.p1_mancala_index] += sum(self.board.board[self.board.p1_pits_index[0]:self.board.p1_pits_index[1]+1])
            self.board.board[self.board.p2_mancala_index] += sum(self.board.board[self.board.p2_pits_index[0]:self.board.p2_pits_index[1]+1])
            
            #Clear the pits
            for i in range(self.board.p1_pits_index[0], self.board.p1_pits_index[1]+1): self.board.board[i] == 0
            for i in range(self.board.p2_pits_index[0], self.board.p2_pits_index[1]+1): self.board.board[i] == 0
            
            self.board.display_board()
            print(self.board.board)
            
            #Determine Winner -- prints
            if self.board.board[self.board.p1_mancala_index] > self.board.board[self.board.p2_mancala_index]:
                print("Player 1 Wins! with : {}".format(self.board.board[self.board.p1_mancala_index]))
            elif self.board.board[self.board.p1_mancala_index] < self.board.board[self.board.p2_mancala_index]:
                print("Player 2 Wins! with : {}".format(self.board.board[self.board.p2_mancala_index]))
            else:
                print("A TIE".format(self.board.board[self.board.p2_mancala_index]))
            return True
        
        #Game NOT over
        return False
            

#Interface 
#To-DO: Finish implementing interface and Game object initialization
class Mancala:
    def __init__(self):
        start_config = self.game_init()
        self.game = Game(*start_config)
        return 
      
    def game_init(self):
        """
        Configures desired Mancala game
        """
        config = self.menu()
        return config       
        
    #TO-DO: implement
    def menu(self):
        #--> TO-DO: UPDATE to include capture of AI toggle... i.e. (PvP, PvCPU, CPUvCPU); 
        # return selected game configuration to game_init
            
        """
        Get player input for game config
        """
        
        #TO DO: Include option for PvP or PvAI!
        print("===Welcome to MANCALA!===")
        print("1. PvP")
        print("2. PvAI")
        print("3. AIvAI")
        print("=========================")
        player_type = int(input("Would you like to PLAY: (PvP) or (PvAI) ... or SPECTATE (AIvAI) :"))
        print("=========================")
        
        if player_type == 1:
            print(f"Awesome! Now playing MANCALA! with Pit Count:(6) and Stone Count(4) with PvP")
            print("")
            print("_____________________________________")
            return (6,4,0)
        if player_type == 2:
            print(f"Awesome! Now playing MANCALA! with Pit Count:(6) and Stone Count(4) with PvAI")
            print("")
            print("_____________________________________")
            return (6,4,1) 
        if player_type == 3:
            print(f"Awesome! Now *SPECTATING* MANCALA! with Pit Count:(6) and Stone Count(4) with AIvAI")
            print("")
            print("_____________________________________")
            return (6,4,2) 


In [80]:
# Manual Game TESTING

##CURRENT ISSUE-->GAME LOGIC IS A LITTLE BROKIES##
# After Round1-2, algo has issues distributing stones into and AFTER mancala indices, whether it's the oppoents or not.

# mancala = Mancala()
# mancala.game.play()

In [81]:
import unittest

In [82]:
#UNIT TEST CLASS RAHHHH
class Test_Mancala_Default(unittest.TestCase):
    
    def test_Mancala_init(self):
        # Test default game setup
        test_mancala = Mancala()
        game_config = test_mancala.game_init()
        self.assertEqual(game_config[0], 6)  # Pits
        self.assertEqual(game_config[1], 4)  # Stones
        self.assertEqual(game_config[2], 0)  # PvP mode

    def test_Game_init(self):
        game = Game(6, 4, 0)
        self.assertEqual(game.board.pits_per_player, 6)
        self.assertEqual(game.board.stones_per_pit, 4)
        self.assertEqual(game.player_mode, 0)

    def test_Board_init(self):
        board = Board(6, 4)
        self.assertEqual(len(board.board), 14)  # 6 pits each + 2 mancalas
        self.assertEqual(board.board[0], 4)     # Stones in first pit
        self.assertEqual(board.board[6], 0)     # Player 1's Mancala
        self.assertEqual(board.board[13], 0)    # Player 2's Mancala
        
class Test_Stone_Distribution(unittest.TestCase):
    def test_distribute_stones(self):
        board = Board(6,4)
        # Player 1 selects pit 1 (4 stones)
        final_index = board.distribute_stones(1,4, board.p2_mancala_index, 1)
        self.assertEqual(final_index, 4) # Target Final Index after distribution
        self.assertEqual(board.board[0], 0)   # Original pit is empty
        self.assertEqual(board.board[1], 5)    # Next pit has one more stone
        self.assertEqual(board.board[4], 5)   # Last stone lands here
                         
    def test_skip_mancala(self):
        board = Board(6,4)
        # Player 1 distributes stones and should skip Player 2's Mancala
        board.board[5] = 10 # Set pit 6 to 10 stones for testing
        final_index = board.distribute_stones(6 , 10, board.p2_mancala_index, 1)
        self.assertNotEqual(final_index, board.p2_mancala_index) # Ensure Player 2's Mancala is skipped
        
class Test_SpecialRules(unittest.TestCase):
    def test_capture_stones(self):
        game = Game(6, 4, 0)
        # Set up a specific board state where the special rule can be tested
        game.current_player = 1
        # Ensure the opposite pit has stones
        game.board.board = [0, 0, 1, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0]
        game.board.display_board()
        # Index mapping:
        # Player 1 pits: indices 0-5
        # Player 1 Mancala: index 6
        # Player 2 pits: indices 7-12
        # Player 2 Mancala: index 13
        # The opposite pit of index 2 (pit 3) is index 9
        game.play_turn(3)
        game.board.display_board()
        # Check that stones are captured correctly
        self.assertEqual(game.board.board[game.board.p1_mancala_index], 6)  # 5 (opponent's stones) + 1 (last stone)
        
        
class Test_MoveValidation(unittest.TestCase):
    def test_valid_move(self):
        board = Board(6,4)
        self.assertTrue(board.valid_move(1,1))   # Pit 1 for Player 1, valid move
        board.board[0] = 0 # Empty Pit
        self.assertFalse(board.valid_move(1,1))  # Cannot select an empty pit
        self.assertFalse(board.valid_move(7,1))  # Cannot select an outside pit range
        
class Test_GameEnd(unittest.TestCase):
    def test_game_end_p1_wins(self):
        game = Game(6,4,0)
        # Empty Player 2's pits to trigger game over
        game.board.board[7:13] = [0,0,0,0,0,0]
        self.assertTrue(game.winning_eval()) 
        self.assertGreater(game.board.board[6], game.board.board[13])
        
    def test_game_end_tie(self):
        game = Game(6,4,0)
        #Equalize Mancala scores to simulate a tie
        game.board.board[6] = 20
        game.board.board[13] = 20
        game.board.board[0:6] = [0, 0, 0, 0, 0, 0] # Empty Player 1's pits
        game.board.board[7:13] = [0, 0, 0, 0, 0, 0] # Empty Player 2's pits

        self.assertTrue(game.winning_eval()) # Game should end
        self.assertEqual(game.board.board[6], game.board.board[13]) # It's a tie
        
        
class Test_Gameplay(unittest.TestCase):
    def test_turn_switch(self):
        game = Game(6, 4, 0)
        game.play_turn(1)  # Player 1 plays pit 1
        self.assertEqual(game.current_player, 0)  # Turn should switch to Player 2

    def test_play_turn(self):
        game = Game(6, 4, 0)
        # Simulate Player 1 playing pit 1
        game.play_turn(1)
        self.assertEqual(game.board.board[0], 0)  # Pit 1 should be empty
        self.assertEqual(game.board.board[1], 5)  # Stones distributed
        self.assertEqual(game.current_player, 0)  # Switch to Player 2

        
        

In [83]:
#Unit testing

#Testing Default --> User input 1, 1, 
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_Mancala_Default))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_Stone_Distribution))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_MoveValidation))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_GameEnd))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_Gameplay))
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Test_SpecialRules))


F
FAIL: test_capture_stones (__main__.Test_SpecialRules)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_623/1743668732.py", line 59, in test_capture_stones
    self.assertEqual(game.board.board[game.board.p1_mancala_index], 6)  # 5 (opponent's stones) + 1 (last stone)
AssertionError: 0 != 6

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (failures=1)


P1               P2
     ____0____     
1 -> | 0 | 0 | <- 6
2 -> | 0 | 0 | <- 5
3 -> | 1 | 0 | <- 4
4 -> | 0 | 5 | <- 3
5 -> | 0 | 0 | <- 2
6 -> |_0_|_0_| <- 1
         0         
Player: 1 chose Pit: (3) with Initial stones: 1
P1               P2
     ____0____     
1 -> | 0 | 0 | <- 6
2 -> | 0 | 0 | <- 5
3 -> | 0 | 0 | <- 4
4 -> | 0 | 5 | <- 3
5 -> | 0 | 0 | <- 2
6 -> |_0_|_0_| <- 1
         0         


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

In [73]:
# 3 rounds Random, continue after with AI
# 
#